In [ ]:
# Function to save cookies
def save_cookies(driver, path):
    with open(path, 'wb') as filehandler:
        pickle.dump(driver.get_cookies(), filehandler)
        
# Get cookie
login_url = "https://sso.teachable.com/secure/212820/identity/login/password"
email = "andrei13@gmail.com"
password = "zPgHR$9AF4byd3MX"
# Path to save or load the cookies
cookies_path_catrill = 'cookies_catrill.pkl'

options = uc.ChromeOptions()
options.headless = False  # Set headless mode
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = uc.Chrome(options=options)
driver.get(login_url)


# Wait until the element is present
element_present = EC.presence_of_element_located((By.XPATH, '/html/body/main/div/form/div[4]/input'))
WebDriverWait(driver, timeout).until(element_present)

 # Locate the email input box and enter the email address
email_input = driver.find_element(By.XPATH, '//*[@id="email"]')
email_input.send_keys(email)
print("Typed email")

# type the password
password_input = driver.find_element(By.XPATH, '//*[@id="password"]')
password_input.send_keys(password)
print("Typed password")

# Locate the lOG IN button and click it
log_in_button = driver.find_element(By.XPATH, '/html/body/main/div/form/div[4]/input')
log_in_button.click()
print("Clicked log_in_button")

#time.sleep(20)  # Wait for all elements to load properly

# Wait until the element is present
element_present = EC.presence_of_element_located((By.XPATH, '//*[@id="navbar"]/div/div/a/img'))
WebDriverWait(driver, timeout).until(element_present)

save_cookies(driver, cookies_path_catrill)
print("Cookies saved.")

In [ ]:
# Navigate to website, navigate all the lectures under url_list
# for each lecture, navigate its logs and get the player urls
# for each player url, navigate its logs and get the vtt files and save them under files_body_responses
# make sure to get an updated cookie before running

#Get player urls, and save vtt files
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import json
import time
import pickle
import os
import re

# Set a timeout duration for the wait
timeout = 30  # seconds

# Set the desired capabilities to include performance logging
caps = DesiredCapabilities.CHROME
caps['goog:loggingPrefs'] = {'performance': 'ALL'}

#options.add_experimental_option('w3c', False)


# Setup
options = uc.ChromeOptions()
options.headless = False
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.set_capability('goog:loggingPrefs', caps['goog:loggingPrefs'])

driver = uc.Chrome(options=options)

#url = "https://learn.cantrill.io/courses/1101194"
url = "https://learn.cantrill.io/courses/2022818"

driver.get(url)
print("Navigated to the course lecture page.")


# Clean string
def process_string(s):
    # Trim leading and trailing whitespaces
    s = s.strip()
    
    # Replace all inner spaces with underscores
    s = s.replace(" ", "_")
    
    # Remove special characters, retain only alphanumeric and underscores
    s = re.sub(r'[^a-zA-Z0-9_]', '', s)
    
    return s

# Load cookies
cookies_path = 'cookies_catrill.pkl'

# Function to load cookies if they exist
def load_cookies(driver, path):
    if os.path.exists(path):
        with open(path, 'rb') as cookiesfile:
            cookies = pickle.load(cookiesfile)
            for cookie in cookies:
                driver.add_cookie(cookie)

# Enable Network monitoring
driver.execute_cdp_cmd('Network.enable', {})

# Load cookies and refresh the page to use them
load_cookies(driver, cookies_path)
driver.refresh()
print(f"Loaded cookies")

#base_url = "https://learn.cantrill.io/courses/aws-certified-developer-associate/lectures/"
base_url = "https://learn.cantrill.io/courses/2022818/lectures/" #Tech Fundamentals
#url_list = ["30553576", "24663362", "39441493", "24663371", "24663376", "24663380", "24663399", "49074937", "32599205", "44031467", "24859593", "24889648", "30339659", "24889649", "24918474", "24918459", "24918469", "24949212", "24949214", "24949219", "24949221", "45685106", "24978287", "24978290", "24978291", "24978331", "24978332", "39551633", "24978333", "24978334", "25213504", "25216284", "25217272", "25301525", "25301526", "25301529", "25301532", "25301533", "25301534", "30489056", "25335801", "25335802", "25335803", "25335804", "25335805", "25335806", "25335807", "40570919", "45250743", "25362687", "25362688", "25362690", "25362691", "25434901", "25527524", "25527525", "30489394", "25870661", "25870662", "25870663", "25870664", "25871102", "25871120", "25996479", "25997309", "25997329", "25997330", "25997332", "46781918", "25997333", "25997334", "25997335", "26227534", "26227536", "26438087", "26438089", "26438090", "30099790", "27895994", "27896001", "27895998", "27896003", "30489615", "27806344", "27994707", "27994708", "27994716", "27994722", "26950363", "26950364", "26953721", "45241152", "26953723", "26953794", "26953800", "26982553", "27186078", "36039667", "26982557", "26982560", "26982639", "26982641", "26982643", "26982655", "27026221", "27806424", "27806425", "27806427", "27806428", "27806430", "29774545", "29774546", "29774547", "29774548", "29774550", "27806440", "27806444", "27806446", "28705524", "28705526", "27806462", "27806464", "27806465", "27806466", "27806467", "27806468", "29064547", "27806469", "27806470", "27806471", "27806472", "27806478", "27806479", "27806480", "27806481", "30490505", "27914996", "27915025", "27915028", "27915458", "46260482", "30614240", "30490576", "27527201", "36184903", "27527204", "27527205", "36185027", "45240218", "42219934", "42220587", "30490580", "27895408", "27895409", "29447330", "27895410", "27895412", "27895413", "27895415", "27895416", "27895417", "29448612", "27895419", "27895420", "30490622", "27415409", "27434132", "27434133", "27451737", "27451738", "27451739", "27451740", "28326940", "28327133", "28327180", "28327217", "28327587", "28331255", "28331425", "28331595", "28332425", "28332455", "28333057", "28333069", "30471883", "28338687", "28338863", "28338903", "29652962", "30490718", "27806928", "27832807", "27832812", "27832822", "27832823", "27832824", "27876488", "30416859", "27832825", "27832826", "27832828", "27832833", "27876494", "27832834", "27887476", "27887477", "27887478", "27887479", "27887480", "30494491", "27887481", "27887482", "29963892", "27887489", "27887490", "27887491", "27887492", "27887493", "27887495", "30490724", "27894828", "27894829", "27894834", "27894839", "27894842", "27894843", "27894844", "27894845", "27894846", "27894847", "27894848", "27894849", "27894850", "27894852", "27894863", "27894865", "27997493", "30490897", "27894723", "27894725", "27894727", "29562476", "27894729", "27894730", "30491014", "27895161", "27895162", "27895165", "27895168", "27895169", "27895171", "27895172", "27895174", "27895176", "27895184", "27895185", "27895186", "27895187", "27895188", "27895189", "27895190", "27895192", "30491100", "27914028", "30415185", "30553573", "30553570", "30553574", "40856877", "40856332", "40856333", "40856334", "40856335", "40856336", "40856337", "40856338", "40856339", "30491783", "27831494", "27831495", "27831502", "27831503", "27831505", "27831561", "27831572", "29902954", "27831596", "27831597", "30318149", "30318150", "30335257", "30335261", "27831598", "27852538", "27831629", "27831630", "27831632", "27831633", "27831634", "27831636", "27831643", "27831644", "27831646", "30268132", "30268150", "30290120", "30290121", "30290123", "30290125", "30491128", "28014597", "30176082", "28014603", "28014604", "30200149", "28014605", "28014607", "30169430", "30169431", "28014613", "28014614", "28014615", "28014616", "28014619", "30237663", "30250385", "30237666", "30237670", "30237671", "45851105", "30237687", "27857414", "27857417", "27857422", "27857423", "27857424", "45004541", "27857425", "27857426", "30335592", "30425831", "45240129", "45240132", "45240141", "30491212", "27896052", "30237855", "30237857", "30237858", "30237859", "30237861", "30237865", "30237867", "30379868", "30380194", "30400048", "30491772", "29929742", "30551240", "30551246", "29929743", "29929745", "29929749", "27026274", "49075047"]
url_list = ["45636987", "45636988", "45700383", "45636946", "45636947", "45636954", "45636948", "45636953", "45636951", "45636950", "45636949", "45661882", "45661883", "45699909", "45699910", "45642191", "45662102", "45641773", "45660745", "45660748", "45660747", "45660746", "45643318", "45662396", "45682862", "45641126", "45641125", "45640673", "45640674", "45637122", "45637121", "45637095", "45637097", "45637092", "45637098", "45637096", "45637093", "45637094", "45637100", "45640670", "45637149", "45642049", "45642050", "45718076", "45718077", "45718075"]
# Ensure the directory for storing response bodies exists
response_dir = 'files_body_responses'
os.makedirs(response_dir, exist_ok=True)

# Define the root directory for logs
json_subdir = "json_files"  # Subdirectory for JSON files
os.makedirs(json_subdir, exist_ok=True)


# Navigate each lecture
for lesson_number in url_list:
    print(f"=================================")
    full_url = base_url + lesson_number
    print(f"Navigating to lecture_url: {full_url}")
    driver.get(full_url)
    
    # Wait for a specific element that indicates the page has fully loaded
    WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.ID, 'lecture_heading')))
    print("Lecture heading is present.")
    
    lesson_title = driver.find_element(By.XPATH, '/html/body/section/div[2]/h2').text
    lesson_title = process_string(lesson_title)
    print(f"lesson_title:{lesson_title}")
    
    # Capture logs and process
    logs = driver.get_log('performance')
    print("Captured performance logs.")

    # For each lecture navigate the logs
    for i, entry in enumerate(logs):
        log_data = json.loads(entry['message'])['message']

        # Find the log that is a player
        if ( log_data['method'] == 'Network.responseReceived' and "params" in log_data.keys()):
            if 'player' in log_data['params']['response']['url']:
                player_url = log_data['params']['response']['url']
                print (f"Navigating to player_url: {player_url}")
                
                # Navigate to the player
                driver.get(player_url)
                
                time.sleep(10)  # Wait for all elements to load properly

                # Capture logs and process in a different log object
                logs_bis = driver.get_log('performance')
                print("Captured performance logs for the player.")
                
                # navigate the logs now for the viewo
                for j, entry_bis in enumerate(logs_bis):
                    log_data_bis = json.loads(entry_bis['message'])['message']

                    try:
                        # Try to find the vtt file in the logs
                        if log_data_bis['method'] == 'Network.responseReceived' and "params" in log_data_bis.keys() and 'vtt' in log_data_bis['params']['response']['url']:
                            print ("Found vtt file")
                            request_id = log_data_bis['params']['requestId']
                            method_name = log_data_bis['method'].split('.')[-1]  # Get the last part of the method name
                            filename = f"log_{request_id}_{method_name}.json"
                            
                            # Define the full path for the file
                            filepath = os.path.join(json_subdir, filename)
                            # Write the log data to a file
                            with open(filepath, 'w') as file:
                                json.dump(log_data_bis, file, indent=4)

                            print(f"json request Log saved to {filepath}")
                            print(log_data_bis['params']['response']['url'])
                            
                            response_body = driver.execute_cdp_cmd("Network.getResponseBody", {"requestId": request_id})
                            response_content = response_body.get('body', '')
                            file_path = os.path.join(response_dir, f'lecture_{lesson_number}_{lesson_title}_{request_id}.txt')
                            
                            with open(file_path, 'w', encoding='utf-8') as file:
                                file.write(response_content)
                            print(f"Response body for request {request_id} saved to {file_path}.")


                    except Exception as e:
                        #print(f"Failed to retrieve response body for request {request_id}")
                        print(f"An Exception occurred : {str(e)}")
                        #pass
driver.quit()


In [28]:
"""
This script extracts course data from an HTML file, 
merges related lecture files from a specified directory
into organized section-based documents, and stores them with descriptive headers.

# update the course_content.html with the html of the course website

"""

import os
import glob
from bs4 import BeautifulSoup

def find_and_combine_lecture_files(base_path, course_data):
    # Ensure the output directory exists
    output_dir = os.path.join(base_path, 'combined_files')
    os.makedirs(output_dir, exist_ok=True)
    
    # Define the path to the specific folder where lecture files are stored
    lecture_files_path = os.path.join(base_path, 'files_body_responses', 'tech fundamentals')
    
    for idx, (section_title, lectures) in enumerate(course_data, start=1):
        # Create a valid filename by removing special characters from the section title
        valid_filename = ''.join(char for char in section_title if char.isalnum() or char.isspace()).replace(' ', '_')
        combined_file_path = os.path.join(output_dir, f"{idx}_{valid_filename}_combined.txt")
        
        # Open the combined file to write
        with open(combined_file_path, 'w', encoding='utf-8') as combined_file:
            for lecture in lectures:
                lecture_id = lecture['id']
                # Use glob to find files matching the pattern
                files_pattern = os.path.join(lecture_files_path, f"lecture_{lecture_id}_*.txt")
                matching_files = glob.glob(files_pattern)
                
                if matching_files:
                    for file_path in matching_files:
                        filename = os.path.basename(file_path)
                        with open(file_path, 'r', encoding='utf-8') as file:
                            content = file.read()
                            combined_file.write(f"--- start of {filename}\n")
                            combined_file.write(content)
                            combined_file.write(f"\n--- end of {filename}\n")
                else:
                    print(f"No files found for lecture ID {lecture_id}.")

    print("All files combined successfully.")

# Assuming the HTML content is in 'course_content.html' in the same base directory
base_path = ''
file_path = os.path.join(base_path, 'course_content.html')

# Read and parse the HTML file
with open(file_path, 'r', encoding='utf-8') as file:
    html_content = file.read()

soup = BeautifulSoup(html_content, 'html.parser')
course_sections = soup.find_all('div', class_='course-section')
course_data = []

for section in course_sections:
    section_title = section.find('div', class_='section-title').get_text(strip=True)
    lectures = section.find_all('li', class_='section-item')
    lecture_details = [{
        'name': lecture.find('span', class_='lecture-name').get_text(strip=True).split('(')[0].strip(),
        'id': lecture['data-lecture-id']
    } for lecture in lectures]
    course_data.append((section_title, lecture_details))

# Function call to find and combine files
find_and_combine_lecture_files(base_path, course_data)


No files found for lecture ID 45636988.
No files found for lecture ID 45700383.
All files combined successfully.
